In [ ]:
# IGNORE NOT GONNA WORK ANYMORE

In [1]:
%reload_ext autoreload
%autoreload 2
from SFM import SFM
import database
import os, shutil, sys
import pycolmap, platform
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [2]:
src = "datasets/templeRing"
dest = "colmap_test_out"
db_path = "test.db"

try:
    os.mkdir(dest)
except:
    shutil.rmtree(dest)
    os.mkdir(dest)

In [3]:
test = SFM(src)
test.ft_extract()
matches = test.ft_match()

pairs.txt: 100%|██████████| 48/48 [00:00<00:00, 121.47it/s]


match.txt is not an img...

pairs.txt is not an img...



FEATURE EXTRACTION....


Image 46: 100%|██████████| 46/46 [00:01<00:00, 32.53it/s]



MATCHING...



(44, 45): 100%|██████████| 45/45 [00:23<00:00,  1.90it/s]                         

Elapsed Time: 23.674301147460938




In [4]:
db = database.COLMAPDatabase.connect(db_path)

In [5]:
db.create_tables()

In [6]:
cam_model, cam_model_value, img_w, img_h, cam_params = test.get_camera_info()
cam_obj = pycolmap.Camera(camera_id=1, model=cam_model_value, width=img_w, height=img_h, params=cam_params)
type(cam_obj)

pycolmap.Camera

In [7]:
# add camera and images

cam = db.add_camera(cam_model_value, img_w, img_h, cam_params)

for file in os.listdir(test.src):
    db.add_image(f"{file}", 1)

db.commit()

In [8]:
# add keypoints and descriptors

kps_per_frame = [np.array([[y.pt[0], y.pt[1], y.size, y.angle] for y in x[0]], dtype=np.float32) for x in test.desc]
desc_per_frame = [x[1] for x in test.desc]

for i, frame_kps in enumerate(kps_per_frame):
    db.add_keypoints(i+1, frame_kps)

for i, frame_desc in enumerate(desc_per_frame):
    db.add_descriptors(i+1, frame_desc)

db.commit()


In [9]:
# add matches and two view
kps = [np.array([[y.pt[0], y.pt[1]] for y in x], dtype=np.float64) for x in test.kp]

for i in matches.keys():
    if isinstance(i, tuple):
        m = np.array([[x.queryIdx,x.trainIdx] for x in matches[i]], dtype=np.uint32)
        if len(m) > 0:
            db.add_matches(i[0]+1, i[1]+1, m)
            t_geo = pycolmap.estimate_calibrated_two_view_geometry(cam_obj, kps[i[0]], cam_obj, kps[i[1]], m).todict()
            db.add_two_view_geometry(
                image_id1 = i[0]+1, 
                image_id2 = i[1]+1, 
                matches = t_geo['inlier_matches'], 
                F = t_geo['F'], 
                E = t_geo['E'],
                H = t_geo['H'],
                config = t_geo['config'])


db.commit()

In [ ]:

# np.set_printoptions(threshold=sys.maxsize)
f = db.execute("""select count(*) from matches""")
f.fetchall()
# [np.array([y for y in x[1]]) for x in test.desc]
# [x[1] for x in test.desc]
# print([np.array([y.pt for y in x[0]], dtype=np.float32) for x in test.desc][0].tobytes())
# print('\n\n\n\n\n\n\n')
# print(np.random.rand(2097, 2) * (504, 672))
# [np.array([[y.pt[0], y.pt[1], y.size, y.angle] for y in x[0]], dtype=np.float32) for x in test.desc]

# for i in test.desc:
#     for j in i[0]:
#         print(j.pt, j.size, j.angle)
# test.output_match()
# [np.array([[y.pt[0], y.pt[1]] for y in x], dtype=np.float32) for x in test.kp][0][0]
np.empty((4, 2), dtype=np.float64).dtype

In [10]:
db.close()

In [ ]:
test.output_match()
pycolmap.match

In [11]:
maps = pycolmap.incremental_mapping(db_path, src, dest)
maps[0].export_PLY(dest + "/object.ply")